# Profiling of terrain corrections

<hr style="border:2px solid blue"> </hr>

### Install Dependencies and Import

Additional dependecies: `sarsen`, `snakeviz`

In [1]:
!pip install -q sarsen snakeviz

In [2]:
%load_ext snakeviz

In [3]:
import os
import tempfile

import adlfs
import planetary_computer
import pystac_client

# enable the `.rio` accessor
import rioxarray  # noqa: F401
import stackstac
from sarsen.apps import terrain_correction

### Processing definitions

In [4]:
# create a temporary directory where to store downloaded data
tmp_dir = tempfile.gettempdir()

# DEM path
dem_path = os.path.join(tmp_dir, "South-of-Redmond-10m.tif")

# path to Sentinel-1 input product in the Planetary Computer
product_folder = "GRD/2021/12/17/IW/DV/S1B_IW_GRDH_1SDV_20211217T141304_20211217T141329_030066_039705_9048"  # noqa: E501

# band to be processed
measurement_group = "IW/VV"

tmp_dir

'/tmp'

#### Area of interest definition: South-of-Redmond (Seattle, US)

In [5]:
lon, lat = [-121.95, 47.04]
buffer = 0.2
bbox = [lon - buffer, lat - buffer, lon + buffer, lat + buffer]

#### DEMs discovery

In [6]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1"
)
search = catalog.search(collections="3dep-seamless", bbox=bbox)
items = list(search.get_items())

In [7]:
# select DEMs with resolution 10 meters
items_high_res = [
    planetary_computer.sign(item).to_dict()
    for item in items
    if item.properties["gsd"] == 10
]

dem_raster_all = stackstac.stack(items_high_res, bounds=bbox).squeeze()
dem_raster_all

/srv/conda/envs/notebook/lib/python3.8/site-packages/stackstac/prepare.py:413: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  xs = pd.Float64Index(np.linspace(minx, maxx, width, endpoint=False))
/srv/conda/envs/notebook/lib/python3.8/site-packages/stackstac/prepare.py:414: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  ys = pd.Float64Index(np.linspace(maxy, miny, height, endpoint=False))


<xarray.DataArray 'stackstac-d0979a9380b3228b778808837b02deea' (time: 4,
                                                                y: 4321, x: 4321)>
dask.array<getitem, shape=(4, 4321, 4321), dtype=float64, chunksize=(1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/13)
  * time             (time) datetime64[ns] 2018-02-02 2018-02-08 ... 2020-01-07
    id               (time) <U10 'n48w122-13' 'n47w123-13' ... 'n48w123-13'
    band             <U4 'data'
  * x                (x) float64 -122.2 -122.1 -122.1 ... -121.8 -121.8 -121.8
  * y                (y) float64 47.24 47.24 47.24 47.24 ... 46.84 46.84 46.84
    start_datetime   (time) <U20 '1952-01-01T00:00:00Z' ... '2017-09-14T00:00...
    ...               ...
    gsd              int64 10
    end_datetime     (time) <U20 '2016-12-31T00:00:00Z' ... '2019-04-25T00:00...
    proj:epsg        int64 5498
    proj:shape       object {10812}
    description      <U1849 'This tile of the 3D Elevation Program (3DEP) sea...
    epsg             int64 5498
Attributes:
    spec:        RasterSpec(epsg=5498, bounds=(-122.15000053746, 46.839907613...
    crs:         epsg:5498
    transform:   | 0.00, 0.00,-122.15|\n| 0.00,-0.00, 47.24|\n| 0.00, 0.00, 1...
    resolution:  9.2592593e-05

#### DEMs average along the time dimension

In [8]:
dem_raster_geo = dem_raster_all.compute()
if "time" in dem_raster_geo.dims:
    dem_raster_geo = dem_raster_geo.mean("time")
_ = dem_raster_geo.rio.set_crs(dem_raster_all.rio.crs)

#### Convert the DEM in UTM coordinates

In [9]:
# find the UTM zone and project in UTM
t_srs = dem_raster_geo.rio.estimate_utm_crs()
dem_raster = dem_raster_geo.rio.reproject(t_srs, resolution=(10, 10))

# crop DEM to our area of interest and save it
dem_corners = dict(x=slice(565000, 594000), y=slice(5220000, 5190000))


dem_raster = dem_raster.sel(**dem_corners)
dem_raster.rio.to_raster(dem_path)
dem_raster

<xarray.DataArray 'stackstac-d0979a9380b3228b778808837b02deea' (y: 3000, x: 2900)>
array([[ 196.68109131,  196.14202881,  194.11082458, ...,  820.71972656,
         820.05755615,  819.68597412],
       [ 196.68109131,  196.14202881,  195.13980103, ...,  817.84838867,
         814.0760498 ,  814.61773682],
       [ 197.03511047,  195.65205383,  195.13980103, ...,  812.12884521,
         810.13116455,  809.48669434],
       ...,
       [ 602.28057861,  607.78918457,  609.63909912, ..., 4130.93261719,
        4135.22167969, 4137.34033203],
       [ 597.96710205,  602.36712646,  603.93676758, ..., 4133.00195312,
        4137.37695312, 4139.43896484],
       [ 594.58496094,  599.09234619,  600.71466064, ..., 4135.22558594,
        4139.52978516, 4141.82373047]])
Coordinates:
  * x                (x) float64 5.65e+05 5.65e+05 ... 5.94e+05 5.94e+05
  * y                (y) float64 5.22e+06 5.22e+06 ... 5.19e+06 5.19e+06
    band             <U4 'data'
    threedep:region  <U7 'n40w130'
    gsd              int64 10
    description      <U1849 'This tile of the 3D Elevation Program (3DEP) sea...
    proj:epsg        int64 5498
    epsg             int64 5498
    proj:shape       object {10812}
    spatial_ref      int64 0
Attributes:
    _FillValue:  1.7976931348623157e+308

### Define GRD parameters

In [10]:
grd_account_name = "sentinel1euwest"
grd_storage_container = "s1-grd"
grd_product_folder = f"{grd_storage_container}/{product_folder}"
grd_local_path = os.path.join(tmp_dir, product_folder)

### Retrieve Sentinel-1 GRD

In [11]:
grd_token = planetary_computer.sas.get_token(
    grd_account_name, grd_storage_container
).token
grd_fs = adlfs.AzureBlobFileSystem(grd_account_name, credential=grd_token)
grd_fs.ls(f"{grd_product_folder}/manifest.safe")

['s1-grd/GRD/2021/12/17/IW/DV/S1B_IW_GRDH_1SDV_20211217T141304_20211217T141329_030066_039705_9048/manifest.safe']

In [12]:
grd_fs.get(grd_product_folder, grd_local_path, recursive=True)
!ls -d {grd_local_path}

/tmp/GRD/2021/12/17/IW/DV/S1B_IW_GRDH_1SDV_20211217T141304_20211217T141329_030066_039705_9048


### Profiling

`%%snakeviz` uses `cProfile` to generate and plot the statistics for profiling the `terrain_correction` functions. If the plots are too large and are not automatically embedded in the notebook, it is possible to visualize the statistics as follows:
* Open a terminal
* Run `snakeviz path/to/profile/stats` (the path to the statistics is displayed in the cell output)
* Use the browser tab opened by `snakeviz` to explore the statistics

#### GTC

In [13]:
%%snakeviz
terrain_correction(
    product_urlpath=grd_local_path,
    measurement_group=measurement_group,
    dem_urlpath=dem_path,
    output_urlpath=os.path.join(
        tmp_dir, os.path.basename(product_folder) + ".10m.GTC.tif"
    ),
)

 
*** Profile stats marshalled to file '/tmp/tmpdhs8tomo'.
Embedding SnakeViz in this document...


#### RTC

##### Nearest neighbour

In [14]:
%%snakeviz
terrain_correction(
    grd_local_path,
    measurement_group=measurement_group,
    dem_urlpath=dem_path,
    correct_radiometry="gamma_nearest",
    output_urlpath=os.path.join(
        tmp_dir, os.path.basename(product_folder) + ".10m.RTC.tif"
    ),
    grouping_area_factor=(3, 3),
)

 
*** Profile stats marshalled to file '/tmp/tmpnoy3n9am'.
Embedding SnakeViz in this document...


##### Bilinear

In [15]:
%%snakeviz
terrain_correction(
    grd_local_path,
    measurement_group=measurement_group,
    dem_urlpath=dem_path,
    correct_radiometry="gamma_bilinear",
    output_urlpath=os.path.join(
        tmp_dir, os.path.basename(product_folder) + ".10m.RTC.tif"
    ),
    grouping_area_factor=(3, 3),
)

 
*** Profile stats marshalled to file '/tmp/tmpwgu4f9bp'.
Embedding SnakeViz in this document...
